# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP.

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset.  

_(That reads the file as a json object per line, which is sort of like a python dictionary.)_

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [2]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [6]:
df.iloc[0]['headline']

'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV'

## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [7]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [8]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

In [11]:
df.iloc[0]['short_description']

'She left her husband. He killed their children. Just another day in America.'

In [10]:
print(data[0])

['There', 'Were', '2', 'Mass', 'Shootings', 'In', 'Texas', 'Last', 'Week', ',', 'But', 'Only', '1', 'On', 'TV', 'She', 'left', 'her', 'husband', '.', 'He', 'killed', 'their', 'children', '.', 'Just', 'another', 'day', 'in', 'America', '.']


Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning. (Recall the vector that captures gender in the previous "king - man + woman = queen" example you saw.)

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [12]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

In [13]:
model

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [14]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55559850, 67352790)

You now have a fully trained model. The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [15]:
wv = model.wv

In [16]:
wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [17]:
wv.most_similar('Texas')

[('Maryland', 0.8148529529571533),
 ('Oregon', 0.8126574754714966),
 ('Louisiana', 0.8102370500564575),
 ('Pennsylvania', 0.807714581489563),
 ('Ohio', 0.797522783279419),
 ('Illinois', 0.7918755412101746),
 ('Massachusetts', 0.7782467007637024),
 ('Georgia', 0.7754024267196655),
 ('Utah', 0.7741360664367676),
 ('Connecticut', 0.7730348110198975)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [23]:
wv.most_similar(negative='Texas')

[('Ex-Lions', 0.3945348560810089),
 ('UIUC', 0.39108070731163025),
 ('Burned-Out', 0.3832785487174988),
 ('Candidate\u200b', 0.3832732141017914),
 ('much-vaunted', 0.37437042593955994),
 ('Slammers', 0.3642950654029846),
 ('Goshen', 0.36173778772354126),
 ('Forty-two', 0.3599264621734619),
 ('once-reliable', 0.3592078685760498),
 ('Shukri', 0.35782963037490845)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [24]:
wv['Texas']

array([-0.01008051, -0.04939622, -0.18387356,  2.362733  ,  2.5945241 ,
        1.3842751 ,  0.75162745,  0.2523451 ,  0.30583328, -0.5702065 ,
        1.010804  ,  0.7003026 ,  0.51031774, -0.8584577 ,  1.7123486 ,
        0.18131645, -1.7068068 , -0.4708611 ,  0.16506505,  0.29589105,
       -0.44351357, -1.2827014 , -0.6734111 , -4.1679144 , -1.4951457 ,
        2.214604  ,  1.7472752 ,  1.0190467 , -0.7366969 , -0.9463645 ,
       -0.58780426,  1.7008772 ,  0.8386665 , -0.7453525 ,  0.28591907,
       -0.06794878,  0.8195547 , -1.5776565 ,  0.3745885 , -1.1449358 ,
       -3.4114687 ,  1.6740084 , -1.9585837 , -0.43235153, -0.4830767 ,
       -0.8236001 ,  0.82068914,  1.7808495 , -2.7076213 , -0.7448797 ,
        2.7728922 ,  0.6353963 ,  1.4154469 ,  1.3091434 ,  0.60355633,
       -1.7477586 ,  0.355193  ,  2.4980412 , -1.0229912 ,  0.58154076,
       -0.22886246, -0.17140596,  0.46107382,  0.800218  ,  1.3385193 ,
       -0.6184762 , -0.9735849 , -2.6256394 ,  1.0363237 , -0.64

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [25]:
wv.vectors

array([[ 8.7105018e-01, -1.6855315e+00, -3.9493528e-01, ...,
        -3.1490833e-01,  6.1760354e-01,  3.9646167e-01],
       [ 5.8410519e-01, -2.2894301e+00, -2.6655018e+00, ...,
        -1.0850933e+00,  1.4401900e+00,  8.6737436e-01],
       [ 8.9663810e-01, -9.0202438e-03, -3.2866502e-01, ...,
        -1.1930332e+00,  6.5505612e-01,  6.0997236e-01],
       ...,
       [-2.3102259e-02,  3.8064984e-03,  5.5694267e-02, ...,
        -6.1475441e-02,  2.8279604e-02, -4.2734172e-02],
       [ 2.0295959e-02, -3.0829692e-02, -4.7054023e-02, ...,
         4.3347839e-02, -2.9591892e-02,  3.0631069e-03],
       [-2.0461474e-03,  3.6281154e-02,  4.7097381e-02, ...,
        -5.5553094e-02, -6.1226770e-02, -5.3066430e-03]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [26]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('princess', 0.6438452005386353),
 ('revival', 0.6149214506149292),
 ('queen', 0.5764859318733215),
 ('goddess', 0.5744208693504333),
 ('lover', 0.5708333253860474),
 ('reminiscent', 0.569614052772522),
 ('diva', 0.5690219402313232),
 ('title', 0.5645473003387451),
 ('epitome', 0.5589678883552551),
 ('theme', 0.5552224516868591)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!

# Playing Around

In [31]:
wv.most_similar('leather')

[('jacket', 0.8403527736663818),
 ('matching', 0.8026828765869141),
 ('purple', 0.801343560218811),
 ('pink', 0.797351598739624),
 ('yellow', 0.797154426574707),
 ('pastel', 0.7912561893463135),
 ('fur', 0.7906641960144043),
 ('velvet', 0.7822037935256958),
 ('shirts', 0.7802686095237732),
 ('lipstick', 0.7772921323776245)]

In [28]:
wv.most_similar('flower')

[('leather', 0.7555139064788818),
 ('yellow', 0.7404510974884033),
 ('jacket', 0.7381378412246704),
 ('purple', 0.7366889119148254),
 ('jewelry', 0.7336482405662537),
 ('pink', 0.7315825819969177),
 ('orange', 0.7267385125160217),
 ('mini', 0.719245195388794),
 ('stone', 0.7154918313026428),
 ('palette', 0.7080853581428528)]

In [34]:
wv.most_similar(positive=['beast', 'flower'])

[('scent', 0.7001819610595703),
 ('yellow', 0.6969966888427734),
 ('ribbon', 0.6941667795181274),
 ('sweater', 0.6710581183433533),
 ('mask', 0.6658969521522522),
 ('bottle', 0.6628925204277039),
 ('pink', 0.6623189449310303),
 ('stone', 0.6568228006362915),
 ('pig', 0.6534896492958069),
 ('jacket', 0.6521004438400269)]

In [37]:
wv.most_similar('silver')

[('flower', 0.6717434525489807),
 ('pink', 0.669437050819397),
 ('diamond', 0.6685705184936523),
 ('gold', 0.6661815643310547),
 ('sweater', 0.6636810898780823),
 ('pair', 0.6549100875854492),
 ('clutch', 0.6432092189788818),
 ('blue', 0.6423872113227844),
 ('stone', 0.6398930549621582),
 ('rock', 0.6366965770721436)]

In [38]:
wv.most_similar('blue')

[('yellow', 0.8273365497589111),
 ('pink', 0.8225677609443665),
 ('gray', 0.7999077439308167),
 ('orange', 0.7776132822036743),
 ('red', 0.7762811779975891),
 ('purple', 0.7591878175735474),
 ('coat', 0.7511742115020752),
 ('bright', 0.7443355321884155),
 ('metal', 0.7438474893569946),
 ('lipstick', 0.7416648864746094)]

In [39]:
wv.most_similar('Obama')

[('Trump', 0.6590725183486938),
 ('Maduro', 0.6585804224014282),
 ('Erdogan', 0.6543865203857422),
 ('Rouhani', 0.654132604598999),
 ('Macron', 0.6339868307113647),
 ('Bush', 0.6305160522460938),
 ('Hollande', 0.6285050511360168),
 ('Ashraf', 0.6203118562698364),
 ('Ghani', 0.6073459982872009),
 ('Netanyahu', 0.603117823600769)]

In [40]:
wv.most_similar(['Obama', 'Trump'])

[('Erdogan', 0.6952388286590576),
 ('Bush', 0.6650077700614929),
 ('Rouhani', 0.6583577990531921),
 ('Macron', 0.6422613859176636),
 ('Maduro', 0.6389861106872559),
 ('Drumpf', 0.628532886505127),
 ('Netanyahu', 0.6148338913917542),
 ('Putin', 0.588276207447052),
 ('Xi', 0.5865577459335327),
 ('Assad', 0.5860471725463867)]

In [38]:
wv.most_similar('Trump')

[('Drumpf', 0.7085047960281372),
 ('Glover', 0.6547946333885193),
 ('Sterling', 0.6542804837226868),
 ('Rumsfeld', 0.6318484544754028),
 ('Ex-President', 0.621883749961853),
 ('Erdogan', 0.6171503067016602),
 ('Trump-Themed', 0.6163948774337769),
 ('Wack', 0.6152186989784241),
 ('deliberation', 0.6126050353050232),
 ('Trump-Joe', 0.6029280424118042)]

In [39]:
wv.most_similar('Obama')

[('Rouhani', 0.6695960760116577),
 ('Macron', 0.6620060801506042),
 ('Erdogan', 0.6420088410377502),
 ('Hollande', 0.6357301473617554),
 ('Maduro', 0.6251469850540161),
 ('Conlin', 0.6242589354515076),
 ('Bush', 0.6190125942230225),
 ('Kosilek', 0.611344575881958),
 ('Xi', 0.6071639060974121),
 ('Ghani', 0.6069828271865845)]

In [40]:
wv.most_similar('Biden')

[('Scarborough', 0.711631178855896),
 ('Manchin', 0.7064214944839478),
 ('Arpaio', 0.6504077911376953),
 ('Isuzu', 0.6503658294677734),
 ('Manganiello', 0.6328834295272827),
 ('Trader', 0.6287012696266174),
 ('Paterno', 0.6213430166244507),
 ('Bush', 0.6182495951652527),
 ('Zee', 0.615400493144989),
 ('G.I', 0.6144809722900391)]

In [41]:
wv.most_similar(positive=['leather', 'jacket'])

[('pink', 0.8303629755973816),
 ('skirt', 0.8216742873191833),
 ('wig', 0.8208991885185242),
 ('purple', 0.814971923828125),
 ('jeans', 0.8141909837722778),
 ('coat', 0.814116895198822),
 ('pastel', 0.8125818371772766),
 ('matching', 0.8104766607284546),
 ('yellow', 0.8045458793640137),
 ('shirt', 0.7968711853027344)]

In [42]:
wv.most_similar(positive=['pink', 'leather', 'jacket'])

[('purple', 0.8489059209823608),
 ('yellow', 0.8482239246368408),
 ('lipstick', 0.8415926098823547),
 ('coat', 0.8343020677566528),
 ('jeans', 0.8314406275749207),
 ('orange', 0.8270695209503174),
 ('pastel', 0.8268184065818787),
 ('skirt', 0.8246691226959229),
 ('matching', 0.819297730922699),
 ('blue', 0.8120771646499634)]

In [43]:
wv.most_similar(positive=['king', 'man'], negative=['queen'])

[('horse', 0.51787930727005),
 ('plane', 0.4913063049316406),
 ('soldier', 0.49064987897872925),
 ('wall', 0.482075035572052),
 ('bird', 0.4711511433124542),
 ('driver', 0.4709591865539551),
 ('tree', 0.4654397964477539),
 ('ride', 0.46329736709594727),
 ('package', 0.4584145247936249),
 ('boy', 0.45487141609191895)]

In [44]:
wv.most_similar(positive=['queen', 'man'], negative=['woman'])

[('king', 0.6416208744049072),
 ('rap', 0.62291419506073),
 ('genius', 0.6192320585250854),
 ('cat', 0.6164597868919373),
 ('princess', 0.6154948472976685),
 ('song', 0.6105555891990662),
 ('charm', 0.5975087881088257),
 ('musician', 0.5956915616989136),
 ('singing', 0.5923466682434082),
 ('darling', 0.5911524295806885)]

In [45]:
wv.most_similar(positive=['king', 'woman'], negative=['king'])

[('man', 0.824468731880188),
 ('person', 0.7840941548347473),
 ('girl', 0.7615827322006226),
 ('teenager', 0.7127897143363953),
 ('bride', 0.6967326402664185),
 ('boy', 0.6762069463729858),
 ('parent', 0.666994035243988),
 ('mother', 0.6410309076309204),
 ('nurse', 0.6358234882354736),
 ('child', 0.6245643496513367)]

In [46]:
wv.most_similar('woman')

[('man', 0.824468731880188),
 ('person', 0.7840942144393921),
 ('girl', 0.7615827322006226),
 ('teenager', 0.7127897143363953),
 ('bride', 0.6967325806617737),
 ('boy', 0.6762069463729858),
 ('parent', 0.666994035243988),
 ('mother', 0.6410309076309204),
 ('nurse', 0.6358234882354736),
 ('child', 0.6245643496513367)]

In [47]:
wv.most_similar(positive=['penis', 'woman'], negative=['man'])

[('hijab', 0.620999813079834),
 ('skirt', 0.5912269949913025),
 ('panties', 0.5795958042144775),
 ('uterus', 0.5773028135299683),
 ('disability', 0.5726682543754578),
 ('vagina', 0.5688541531562805),
 ('wheelchair', 0.5683301687240601),
 ('waist', 0.5654455423355103),
 ('diaper', 0.5637011528015137),
 ('bandmate', 0.5635241866111755)]